In [128]:
from torchvision.datasets import CelebA
import os
import torch.nn as nn
import glob
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import math
# main libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset
import math
import torchvision.transforms.functional as TF
import cv2
from keras.utils import image_dataset_from_directory
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.nn.parameter import Parameter

from torch.nn import init
from torchvision.utils import make_grid
from collections import OrderedDict

import torch
from torch.utils.data import Dataset

from skimage import transform
import skimage.io as io
import numpy as np
from torchvision.utils import save_image
import torch
from torch import nn
from torchvision.models import vgg16
from pathlib import Path
from torch.autograd import Variable

In [184]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features

        #################################################################
        # GRAD FALSE #
        #################################################################
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")
        #################################################################



        # ASLE KAR
        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        #################################################################
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))
        bottleneck = self.bottleneck(self.pool4(enc4))
        #################################################################


        bottleneck = torch.empty_like(bottleneck).normal_(0,1)
        dec4 = self.upconv4(bottleneck)

        enc4 = torch.empty_like(enc4).normal_(0,1)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)

        enc3 = torch.empty_like(enc3).normal_(0,1)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)

        enc2 = torch.empty_like(enc2).normal_(0,1)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)

        enc1 = torch.empty_like(enc1).normal_(0,1)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.tanh(self.conv(dec1))




    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [187]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [188]:
unet = UNet(1,1,8).cuda()
count_parameters(unet)

486409

In [193]:
unet = UNet(1,1,16).cuda()
unet.get_submodule('encoder1').requires_grad_(False)
unet.get_submodule('encoder2').requires_grad_(False)
unet.get_submodule('encoder3').requires_grad_(False)
unet.get_submodule('encoder4').requires_grad_(False)
unet.get_submodule('bottleneck').requires_grad_(False)

count_parameters(unet)

762817

In [194]:
# TEST

x = torch.ones(10, 1, 128, 128).to(torch.float).cuda()
unet(x).shape

torch.Size([10, 1, 128, 128])

In [ ]:
class Layer(nn.Module):
    def __init__(self, latent_dim, features, init_size):
        super(Layer, self).__init__()


        self.z0_2_w0 = nn.Sequential(nn.Linear(latent_dim, features * init_size ** 2))

        nn.Sequential(
            nn.BatchNorm2d(features),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(features, features, 3, stride=1, padding=1)
        )


In [40]:
nn.Upsample(2)(torch.ones(10,128,1,1)).shape

torch.Size([10, 128, 2, 2])

In [58]:
class NonStationary(nn.Module):
    """
    # (N, latent_dim)
    # (N, features, 2*H0, 2*W0)
    """
    def __init__(self, latent_dim, in_channels, features, H0, W0):
        super(NonStationary, self).__init__()

        self.H0 = H0
        self.W0 = W0
        self.in_channels = in_channels

        self.linear_layer = nn.Linear(latent_dim, in_channels * H0 * W0)

        self.conv_layers = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(in_channels, features, 3, stride=1, padding=1)
        )

    def forward(self, z):
        w = self.linear_layer(z).view(-1, self.in_channels, self.H0, self.W0)
        return self.conv_layers(
                w
            )

In [45]:
# TEST
m = MyModule(100, 16, 8, 1, 1)
z = torch.ones(10, 100)
m(z).shape

torch.Size([10, 8, 2, 2])

In [59]:
class MyGenerator(nn.Module):
    def __init__(self, latent_dim, channels, H0, W0):
        super(MyGenerator, self).__init__()

        self.gen = nn.Sequential(
            NonStationary(latent_dim, 128, 128, H0, W0),
            MyGenerator._block(128, 128),
            MyGenerator._block(128, 64),
            MyGenerator._block(64, channels)
        )

    def forward(self, z):
        return self.gen(z)


    @staticmethod
    def _block(in_channels, features):
        return nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(in_channels, features, 3, stride=1, padding=1)
        )

In [65]:
# TEST
m = MyGenerator(100, 1, 8, 8)
z = torch.ones(10, 100)
m(z).shape

torch.Size([10, 1, 128, 128])

In [16]:
class Generator(nn.Module):
    def __init__(self, img_size, latent_dim, channels):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.linear_layer = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_layers1 = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1)
        )

        self.conv_layers2 = nn.Sequential(
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
        )

        self.conv_layers3 = nn.Sequential(
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )


    def forward(self, z):
        out = self.linear_layer(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        self.out1 = self.conv_layers1(out)
        self.out2 = self.conv_layers2(self.out1)
        self.out3 = self.conv_layers3(self.out2)
        return self.out3

In [ ]:
torch.empty_like()

In [17]:
z = torch.FloatTensor(np.random.normal(0, 1, (10, 100))).cpu()
gen = Generator(96, 100, 1)
gen(z).shape

torch.Size([10, 1, 96, 96])

In [20]:
gen.out3.shape

torch.Size([10, 1, 96, 96])